In [ ]:
import numpy as np
import matplotlib.pyplot as plt

from avapi.carla import CarlaScenesManager

data_root = "/data/shared/CARLA/multi-agent-intersection"
CSM = CarlaScenesManager(data_dir=data_root, split="train")
CDM = CSM.get_scene_dataset_by_index(0)

In [ ]:
from avstack.modules.perception.fov_estimator import FastRayTraceBevLidarFovEstimator
from avstack.modules.perception.fov_estimator import SlowRayTraceBevLidarFovEstimator

# set up fov models
fov_estimator_fast = FastRayTraceBevLidarFovEstimator(z_min=-3.0, z_max=3.0)
fov_estimator_slow = SlowRayTraceBevLidarFovEstimator(z_min=-3.0, z_max=3.0)

# get some random lidar point clouds to analyze
seed = 10
rng = np.random.RandomState(seed=seed)
n_pcs = 20
pcs = []
frames = CDM.get_frames(sensor="lidar-0", agent=0)
random_agent_frames = list(zip(rng.randint(0, 4, size=n_pcs), rng.choice(list(range(200)), size=n_pcs, replace=False)))
pcs = [CDM.get_lidar(frame=frames[idx_frame], sensor="lidar-0", agent=agent) for agent, idx_frame in random_agent_frames]

In [ ]:
for pc in pcs:
    # run fov estimator on this frame
    fov_hull = pc.concave_hull_bev(concavity=1)
    fov_fast = fov_estimator_fast(pc)
    fov_slow = fov_estimator_slow(pc)
    
    # project lidar to bev for plotting
    pc_bev = pc.project_to_2d_bev()
    
    # make plot
    plt.plot(fov_hull.boundary[:,0], fov_hull.boundary[:,1], "r-", label="Hull FOV Model")
    plt.plot(fov_fast.boundary[:,0], fov_fast.boundary[:,1], "k--", label="Fast FOV Model")
    plt.plot(fov_slow.boundary[:,0], fov_slow.boundary[:,1], "g-", label="Slow FOV Model")
    plt.plot(pc_bev.data.x[:,0], pc_bev.data.x[:,1], "bo", markersize=2, label="Point Cloud")
    plt.xlabel("X")
    plt.ylabel("Y")
    plt.legend()
    plt.show()

In [ ]:
# run execution timing check
n_pcs_timing = 10
%timeit for pc in pcs[:n_pcs_timing]:fov_estimator_fast(pc)
%timeit for pc in pcs[:n_pcs_timing]:fov_estimator_slow(pc)

## Run a simple unit test

In [ ]:
from avstack.geometry import Position, ReferenceFrame, GlobalOrigin3D

def make_far_reference():
    return ReferenceFrame(
        x=np.array([100, 100, 0]), q=np.quaternion(1), reference=GlobalOrigin3D
    )


pc_local = CDM.get_lidar(frame=frames[0], sensor="lidar-0", agent=0)

# run fov estimator in a simple frame
fov_estimator = FastRayTraceBevLidarFovEstimator()
fov_local = fov_estimator(pc_local, in_global=False)
pt_local = Position(np.array([-20,0,0], dtype=float), reference=pc_local.reference)
assert fov_local.check_point(pt_local.x)

# convert the fov frame using change reference
pc_far = pc_local.change_calibration(make_far_reference(), inplace=False)
pt_far = pt_local.change_reference(pc_far.reference, inplace=False)
fov_local_as_far = fov_local.change_reference(pc_far.reference, inplace=False)
assert fov_local_as_far.check_point(pt_far.x)

# run fov estimator in a different frame
fov_far = fov_estimator(pc_far, in_global=False)
assert fov_far.check_point(pt_far.x)

In [ ]:
# show in a local frame
plt.plot(pc_local.data.x[:,0], pc_local.data.x[:,1], "bo", alpha=0.2, markersize=2, label="Point Cloud")
plt.plot(fov_local.boundary[:,0], fov_local.boundary[:,1], "k--", label="Fast FOV Model")
plt.plot(pt_local.x[0], pt_local.x[1], "gx", markersize=10, label="point")
plt.show()

# show in the transformed frame
plt.plot(pc_far.data.x[:,0], pc_far.data.x[:,1], "bo", alpha=0.2, markersize=2, label="Point Cloud")
plt.plot(fov_local_as_far.boundary[:,0], fov_local_as_far.boundary[:,1], "k--", label="Fast FOV Model")
plt.plot(pt_far.x[0], pt_far.x[1], "gx", markersize=10, label="point")
plt.show()

# show in the far frame
plt.plot(pc_far.data.x[:,0], pc_far.data.x[:,1], "bo", alpha=0.2, markersize=2, label="Point Cloud")
plt.plot(fov_far.boundary[:,0], fov_far.boundary[:,1], "k--", label="Fast FOV Model")
plt.plot(pt_far.x[0], pt_far.x[1], "gx", markersize=10, label="point")
plt.show()